In [ ]:
import math
import nibabel as nib
import numpy as np
from numpy.fft import fft, fft2, ifft, fftshift, ifftshift
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import butter, lfilter, freqz, filtfilt
from scipy.io import loadmat

### Figure

In [ ]:
# Load cylinder
susc = nib.load(os.path.join("field_dist", "cylinder", "Chi.nii")).get_fdata()
fmap_hz_all = nib.load(os.path.join("field_dist", "cylinder", "fmap_hz.nii")).get_fdata()
local_field_cyl = nib.load(os.path.join("field_dist", "cylinder", "local_field.nii")).get_fdata()

# Load brain
susc_brain = nib.load(os.path.join("field_dist", "brain", "chi_masked.nii")).get_fdata()
fmap_hz_brain_all = nib.load(os.path.join("field_dist", "brain", "fmap_hz.nii")).get_fdata()
local_field_brain = nib.load(os.path.join("field_dist", "brain", "local_field.nii")).get_fdata()

fig = make_subplots(rows=2, cols=3, shared_xaxes=False, horizontal_spacing=0.1, subplot_titles=("Susceptibility distribution (ppm)", "Simulated B0 map (Hz)", "Simulated B0 map (Hz), no background field", "Susceptibility distribution (ppm)", "Simulated B0 map (Hz)", "Simulated B0 map (Hz), no background field"), specs=[[{"type": "Heatmap"}, {"type": "Heatmap"}, {"type": "Heatmap"}], [{"type": "Heatmap"}, {"type": "Heatmap"}, {"type": "Heatmap"}]], )
fig.add_trace(go.Heatmap(z=susc[:,64,:], colorscale='gray', colorbar_x=1/3 - 0.05, colorbar_y=0.8, colorbar_len=0.4))
fig.add_trace(go.Heatmap(z=fmap_hz_all[:,64,:], colorscale='gray', colorbar_x=2/3 - 0.01, colorbar_y=0.8, colorbar_len=0.4), 1, 2)
fig.add_trace(go.Heatmap(z=local_field_cyl[:,64,:], colorscale='gray', colorbar_y=0.8, colorbar_len=0.4), 1, 3)
fig.add_trace(go.Heatmap(z=np.rot90(susc_brain[:,:,162], k=-1), colorscale='gray', colorbar_x=1/3 - 0.05, colorbar_y=0.2, colorbar_len=0.4), 2, 1)
fig.add_trace(go.Heatmap(z=np.rot90(fmap_hz_brain_all[:,:,162], k=-1), colorscale='gray', colorbar_x=2/3 - 0.03, colorbar_y=0.2, colorbar_len=0.4, zmin=535, zmax=714), 2, 2)
fig.add_trace(go.Heatmap(z=np.rot90(local_field_brain[:,:,162], k=-1), colorscale='gray', zmin=-4, zmax=4, colorbar_y=0.2, colorbar_len=0.4), 2, 3)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.update_layout({"height": 1000})
fig.show()


### Figure

In [ ]:
fname_mag_e1 = os.path.join("fmap", "BR_e1_M.nii")
fname_phase_e1 = os.path.join("fmap", "BR_e1_ph.nii")
fname_phase_e2 = os.path.join("fmap", "BR_e2_ph.nii")
fname_phase_e3 = os.path.join("fmap", "BR_e3_ph.nii")
fname_phase_e4 = os.path.join("fmap", "BR_e4_ph.nii")
fname_mask = os.path.join("fmap", "mask.nii.gz")
fname_fmap = os.path.join("fmap", "fmap.nii")

nii_mag_e1 = nib.load(fname_mag_e1)
nii_phase_e1 = nib.load(fname_phase_e1)
nii_phase_e2 = nib.load(fname_phase_e2)
nii_phase_e3 = nib.load(fname_phase_e3)
nii_phase_e4 = nib.load(fname_phase_e4)
nii_mask = nib.load(fname_mask)
nii_fmap = nib.load(fname_fmap)


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
n=3
# Attempt at subplots
fig = make_subplots(rows=1, cols=n, shared_xaxes=False, horizontal_spacing=0.1, subplot_titles=("Magnitude", "Phase (rad)", "B0 Field map (Hz)"), specs=[[{"type": "Heatmap"}, {"type": "Heatmap"},
           {"type": "Heatmap"}]], )

fig.add_trace(go.Heatmap(z=np.rot90(nii_mag_e1.get_fdata()[:-10,:,50], k=-1), colorscale='gray', colorbar_x=1/n - 0.05), 1, 1)
fig.add_trace(go.Heatmap(z=np.rot90(nii_phase_e4.get_fdata()[:-10,:,50] / 4095 * 2 * math.pi - math.pi, k=-1), colorscale='gray', colorbar_x=2/n - 0.02), 1, 2)
fig.add_trace(go.Heatmap(z=np.rot90(nii_fmap.get_fdata()[:-10,:,50], k=-1), colorscale='gray'), 1, 3)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

### Figure

In [ ]:
import numpy as np
import plotly.graph_objs as go

# Parameters
num_frames = 50
omega_0 = 1  # Larmor frequency
omega_1 = 0.9  # Inhomogeneous spin

# Initial phase of the spin
# initial_phase = np.random.uniform(0, 2*np.pi)
initial_phase = 0.5

# Time array
time = np.linspace(0, 1, num_frames)

# Generate data for spins
x = np.cos(omega_0 * (2*math.pi) * time + initial_phase)
y = np.sin(omega_0 * (2*math.pi) * time + initial_phase)
x1 = np.cos(omega_1 * (2*math.pi) * time + initial_phase)
y1 = np.sin(omega_1 * (2*math.pi) * time + initial_phase)

angles = (np.arctan2(y,x))
angles1 = (np.arctan2(y1,x1))

# Create figure
fig = make_subplots(rows=1, cols=2, shared_xaxes=False, horizontal_spacing=0.1, subplot_titles=("Spin Rotating Through Time", "Phase Evolution of the Signal (rad)"))

# Add spin as an arrow
fig.add_trace(go.Scatter(
    x=[0, x[0]],
    y=[0, y[0]],
    mode='lines+markers',
    marker=dict(size=5),
    line=dict(color='blue', width=3),
    name='MRI Spin'),
    row=1, col=1)
fig.add_trace(go.Scatter(
    x=[0, x1[0]],
    y=[0, y1[0]],
    mode='lines+markers',
    marker=dict(size=5),
    line=dict(color='red', width=3),
    name='Inhomogeneous MRI Spin'),
    row=1, col=1)

# Add phase of the signal
fig.add_trace(go.Scatter(
    x=[time[0]],
    y=[angles[0]],
    mode='lines+markers',
    marker=dict(color='blue', size=5),
    name='Phase'),
    row=1, col=2
)
fig.add_trace(go.Scatter(
    x=[time],
    y=[angles1],
    mode='lines+markers',
    marker=dict(color='red', size=5),
    name='Inhomogeneous Phase'),
    row=1, col=2
)

fig.update_xaxes(range=[-1.1, 1.1], row=1, col=1)
fig.update_yaxes(range=[-1.1, 1.1], row=1, col=1)
fig.update_xaxes(range=[np.min(time), np.max(time)], row=1, col=2)
fig.update_yaxes(range=[np.min(angles) + 0.1*np.min(angles), np.max(angles) + 0.1*np.max(angles)], row=1, col=2)

# Add frames
frames = [dict(
    data=[go.Scatter(x=[0, x[i]],
                     y=[0, y[i]],
                     mode='lines+markers',
                     marker=dict(size=5),
                     line=dict(color='#636EFA', width=3),
                     name='MRI Spin'),
          go.Scatter(x=[0, x1[i]],
                     y=[0, y1[i]],
                     mode='lines+markers',
                     marker=dict(size=5),
                     line=dict(color='#fa6363', width=3),
                     name='Inhomogeneous MRI Spin'),
          go.Scatter(x=time[:i],
                     y=angles[:i],
                     mode='lines+markers',
                     marker=dict(size=5),
                     line=dict(color='blue', width=3),
                     name='Phase'),
          go.Scatter(x=time[:i],
                     y=angles1[:i],
                     mode='lines+markers',
                     marker=dict(size=5),
                     line=dict(color='red', width=3),
                     name='Inhomogeneous Phase')],
    name=str(i),
    traces=[0,1,2,3]) for i in range(num_frames)]

fig.frames = frames

# Determine the maximum absolute value of coordinates
max_coord = max(abs(x.max()), abs(y.max()))

fig.update_xaxes(title_text="x", row=1, col=1)
fig.update_xaxes(title_text="time", row=1, col=2)
fig.update_yaxes(title_text="y", row=1, col=1)
fig.update_yaxes(title_text="rad", tickmode = 'array',
        tickvals = [-math.pi, math.pi],
        ticktext = ['-\u03C0', '\u03C0'], row=1, col=2)

# Update layout
fig.update_layout(
    xaxis=dict(autorange=False),
    yaxis=dict(autorange=False),
    updatemenus=[dict(
        type='buttons',
        buttons=[dict(label='Play',
                      method='animate',
                      args=[None, dict(frame=dict(duration=100, redraw=True), fromcurrent=True)]),
                 dict(label='Pause',
                      method='animate',
                      args=[[None], dict(frame=dict(duration=0, redraw=True), mode='immediate')])])])

# Show figure
fig.show()

### Figure

In [ ]:
# Note: Field was reduced a lot to be able to show the sinusoid
# Note: The inhomogeneities are small, to visually show y_1, it is ~10%, units do not make much sense as it is after demodulation
GYRO_BAR_RATIO_H = 42.6e6  # [Hz/T]
b0 = 0.000002  # [T]
T2 = 0.3  # s
y_0_cst = 10
fs = 10000

f_larmor = b0 * GYRO_BAR_RATIO_H
t = np.linspace(0, 1, fs + 1)  # 1 second

def butter_lowpass(cutoff, fs, order=5):
    return butter(order, cutoff, fs=fs, btype='low', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data, method='gust')
    return y

# Lab frame
y_0 = y_0_cst * np.sin(2 * math.pi * f_larmor * t)
exp = np.exp(-t/T2)
y = y_0 * exp / y_0_cst
temp = y * (y_0 / y_0_cst)
y_demod = butter_lowpass_filter(temp, f_larmor, fs, order=5) * 2

fig = go.Figure()
fig.add_scatter(x=t, y=y, name="FID")
fig.add_scatter(x=t, y=y_demod, name="Demodulated FID")
fig.add_scatter(x=t, y=exp, name="T2 decay")
fig.update_traces(marker=dict(size=3))
fig.update_layout(title="One isochromat at Larmor frequency")
fig.show()
# y_fft = np.log(np.abs(fftshift(fft(ifftshift(y)))))
# fig = px.scatter(x=t, y=y_fft)
# fig.update_traces(marker=dict(size=3))
# fig.show()

# 2nd isochromat
y_1 = np.sin(2 * math.pi * (f_larmor + 10) * t)
y = (y_0 + y_1) * exp / (y_0_cst + 1)
temp = y * (y_0 / y_0_cst)
y_demod = butter_lowpass_filter(temp, f_larmor, fs, order=5) * 2
fig = go.Figure()
fig.add_scatter(x=t, y=y, name="FID")
fig.add_scatter(x=t, y=y_demod, name="Demodulated FID")
fig.add_scatter(x=t, y=exp, name="T2 decay")
fig.update_traces(marker=dict(size=3))
fig.update_layout(title="Two isochromats, one at Larmor, one off frequency")
fig.show()
# y_fft = np.log(np.abs(fftshift(fft(ifftshift(y)))))
# fig = px.scatter(x=t, y=y_fft)
# fig.update_traces(marker=dict(size=3))
# fig.show()

# Multiple isochromats
n_freqs = 30
y = y_0
for i in range(n_freqs):
    amp = 10
    scale = amp/n_freqs
    mid = n_freqs // 2
    y_1 = np.sin(2 * math.pi * (f_larmor + scale*(mid - i)) * t)
    y = y + y_1
y = y * exp / (y_0_cst + n_freqs)
temp = y * (y_0 / y_0_cst)
y_demod = butter_lowpass_filter(temp, f_larmor, fs, order=5) * 2
fig = go.Figure()
fig.add_scatter(x=t, y=y, name="FID")
fig.add_scatter(x=t, y=y_demod, name="Demodulated FID")
fig.add_scatter(x=t, y=exp, name="T2 decay")
fig.update_traces(marker=dict(size=3))
fig.update_layout(title="Inhomogeneous sample")
fig.show()
# y_fft = np.log(np.abs(fftshift(fft(ifftshift(y)))))
# fig = px.scatter(x=t, y=y_fft)
# fig.update_traces(marker=dict(size=3))
# fig.show()